# Create IPO Master Table 

In [1]:
#import library

import yfinance as yf
import pandas as pd, numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("IPO/ipo_calendar.csv")
df.head()

,IPODate,CompanyName,StockCode,IPOValue,IPOPrice
0,2019-01-31,New Fortress Energy LLC,NFE,300.0M,15
1,2019-01-31,Pivotal Acquisition Corp,PVT_u,200.0M,10
2,2019-02-28,Super League Gaming Inc,SLGG,27.3M,10.00-12.00
3,2019-02-28,Tortoise Acquisition,SHLL_u,225.0M,10
4,2019-02-27,Kaleido Biosciences Inc,KLDO,104.8M,20.00-22.00


In [3]:
df[df.StockCode.isna()]

,IPODate,CompanyName,StockCode,IPOValue,IPOPrice
1488,2022-07-12,Nano Labs Ltd,NaN,20.4M,11.5


In [4]:
survivors = []

for code in list(df.StockCode.values):
    if type(code) == str:
        if code[-1] != "u":
            survivors.append(code)
            
    # Nano Labs -> NA        
    else:
        # only for Nano Labs
        survivors.append("NA")

survivors[:10]

['NFE', 'SLGG', 'KLDO', 'SOLY', 'MITO', 'HOTH', 'TCRR', 'AVDR', 'ANCN', 'BFL']

In [5]:
IPO = df.loc[df.StockCode.isin(survivors)]
IPO = IPO.reset_index(drop=True)
IPO.head()

,IPODate,CompanyName,StockCode,IPOValue,IPOPrice
0,2019-01-31,New Fortress Energy LLC,NFE,300.0M,15
1,2019-02-28,Super League Gaming Inc,SLGG,27.3M,10.00-12.00
2,2019-02-27,Kaleido Biosciences Inc,KLDO,104.8M,20.00-22.00
3,2019-02-19,Soliton,SOLY,10.9M,5
4,2019-02-15,Stealth BioTherapeutics Inc,MITO,86.8M,12.00-14.00


In [ ]:
# get IPO data from yfinance

tickers = list(IPO.StockCode.values)
start_date = IPO.IPODate.min()
end_date = IPO.IPODate.max()

IPOraw = yf.download(tickers, start_date, end_date)

[**********************78%************           ]  1090 of 1400 completed

In [ ]:
#sorted(tickers)

In [ ]:
IPOdata = IPOraw["Adj Close"].dropna(axis=1, how='all') # drop if all values = NaN
#IPOdata.columns.name = ""
IPOdata.head()

In [ ]:
IPOdata.shape

In [ ]:
ticker_dic = {}

max_length = IPOdata.shape[0]

for col in IPOdata.columns:
    L = [round(p,4) for p in IPOdata[col].values if str(p) != "nan"]
    
    if len(L) < max_length:
        addition = max_length - len(L)
        L = L + ["ND"]*addition
    
    ticker_dic[col] = L

list(ticker_dic.keys())[:10]

In [ ]:
ticker_dic["AMPL"][-10:]

In [ ]:
master = pd.DataFrame(ticker_dic).T
master.head()

In [ ]:
performance_dic = {"ticker":[],
                   "start_price":[],
                   "x1.5_price":[],
                   "x1.5_days":[],
                   "x2_price":[],
                   "x2_days":[],
                   "x3_price":[],
                   "x3_days":[],
                   "max_price":[],
                   "max_ratio":[],
                   "max_days":[],
                  }


for index in master.index:
    start = master.at[index,0]
    
    prices = ticker_dic[index]
    
    m15 = ("DNE","DNE")
    m2 = ("DNE","DNE")
    m3 = ("DNE","DNE")
    
    mm = (0,"DNE")
    
    for i in range(max_length):
        if prices[i] == "ND":
            break
        else:
            if prices[i] >= mm[0]:
                mm = (prices[i], i)    
        
            if prices[i] >= start*3:
                if m3[0] == "DNE":
                    m3 = (prices[i], i)
                    
            if prices[i] >= start*2:
                if m2[0] == "DNE":
                    m2 = (prices[i], i)
                    
            if prices[i] >= start*1.5:
                if m15[0] == "DNE":
                    m15 = (prices[i], i) 
       
    performance_dic["ticker"].append(index)
    performance_dic["start_price"].append(start)
    
    performance_dic["x1.5_price"].append(m15[0])    
    performance_dic["x1.5_days"].append(m15[1])

    performance_dic["x2_price"].append(m2[0])    
    performance_dic["x2_days"].append(m2[1])

    performance_dic["x3_price"].append(m3[0])
    performance_dic["x3_days"].append(m3[1])

    performance_dic["max_days"].append(mm[0])
    performance_dic["max_ratio"].append(mm[1]/start)
    performance_dic["max_price"].append(mm[1])


In [ ]:
performance = pd.DataFrame(performance_dic)
performance.tail()

In [ ]:
performance[performance["x1.5_price"] != "DNE"]

In [ ]:
performance[performance["x1.5_price"] != "DNE"]["x1.5_price"].mean()

In [ ]:
performance[performance["x1.5_price"] != "DNE"]["x1.5_days"].mean()

In [ ]:
performance[performance["x2_price"] != "DNE"]

In [ ]:
performance[performance["x2_price"] != "DNE"]["x2_price"].mean()

In [ ]:
performance[performance["x2_price"] != "DNE"]["x2_days"].mean()

In [ ]:
performance[performance["x3_price"] != "DNE"]

In [ ]:
performance[performance["x3_price"] != "DNE"]["x3_price"].mean()

In [ ]:
performance[performance["x3_price"] != "DNE"]["x3_days"].mean()

In [ ]:
performance.max_ratio.median(), performance.max_days.median(), performance.max_price.median()

In [ ]:
performance.max_ratio.mean(), performance.max_days.mean(), performance.max_price.mean()